In [1]:
import os
import cobra
import scripts.io
import scripts.gen_model
import scripts.reaction_utils
import scripts.metabolite_utils
import math
import numpy as np
import importlib
import thermo_flux.io.load_sbml


from thermo_flux.tools.drg_tools import reaction_balance

/home/laurens/miniconda3/envs/thermo_pta/lib/python3.11/site-packages/enkie/compartment_parameters.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
INPUT_MODEL = "datafiles/model.xlsx"
INPUT_KEGGS = "datafiles/ecoli_kegg_id.csv"
INPUT_REED = "regression/reed.csv"
INPUT_INCHI = "regression/InChIs.csv"
INPUT_GAMS = "regression/model_Ecoli_from-gams.xlsx"
INPUT_EXP_DATA = "regression/allPhysioData_formatted_forGSM_20230831.csv"
INPUT_EXP_CONC = "regression/allConcRange_20230912.csv"
INPUT_METABOLOMICS = "regression/metabolomics-Kochanowski_20230925.csv"

MODEL_NAME = "ecoli"

CONDITIONS = ["WT-Glc_I", "WT-Ace_I"]

INCLUDE_CO2 = True
INCLUDE_O2 = True
ALLOW_OTHER_EXCRETION = False
RELAX_EXP_FLUX_BOUNDS = 2.0

In [3]:
import pandas as pd

def add_rq_constraint(tmodel, condition, phys_data):
    
    reg_data = pd.read_csv(phys_data)

    reg_data.set_index(["cond", "rxn"], inplace=True) 

    reg_data_gas = reg_data.swaplevel().copy()
    reg_data_gas = reg_data_gas.loc[["EX_co2", "EX_o2"]]
    reg_data_gas = reg_data_gas.swaplevel()

    data_gas = reg_data_gas.loc[condition]
    vo2, vo2_err = data_gas.loc["EX_o2"]
    vco2, vco2_err = data_gas.loc["EX_co2"]
    rq = - vco2 / vo2
    rq_err = rq * np.sqrt( (vo2_err / vo2)**2 + (vco2_err / vco2)**2)
    print(f">>>>>> RQ: {rq :.2} (vCO2 = {vco2 :.3} / vO2 = {vo2 :.3}) ERR: {rq_err}")

    rq_lb = rq - abs(6 * rq_err)
    rq_ub = rq + abs(6 * rq_err)

    #resrq = tmodel.m.addMVar(lb=rq_lb, ub=rq_ub, shape=(1,1), name="resRQ") 
    mrq = tmodel.m.addMVar(lb=rq_lb, ub=rq_ub, shape=(1,1), name="RQ") 
    #tmodel.mvars["resRQ"] = resrq
    tmodel.mvars["rq"] = mrq
    
    #tmodel.m.addConstr(tmodel.mvars["resRQ"][0, 0] >= ( (mrq - rq)/rq_err ), name = "resRQ_pos")
    #tmodel.m.addConstr(tmodel.mvars["resRQ"][0, 0] >= (-(mrq - rq)/rq_err ), name = "resRQ_neg")

    idx_o2 = tmodel.reactions.index("EX_o2")
    idx_co2 = tmodel.reactions.index("EX_co2")
    tmodel.m.addConstr(tmodel.mvars["v"][0, idx_co2] == (-mrq)*tmodel.mvars["v"][0, idx_o2], name="enforce_RQ")
    tmodel.m.update()


In [ ]:
def apply_tfva_bounds(tmodel, tfva_file):
    bounds = []
    with open(tfva_file, "r") as f:
        for line in f:
            tokens = line.strip().split("\t")
            bounds.append( (int(tokens[0]), float(tokens[1]), float(tokens[2])) )

    for idx, lb, ub in bounds:

        lb, ub = clean_tfva_bounds(lb, ub)
        print(f"{tmodel.reactions[idx].id}: {tmodel.reactions[idx].lower_bound}, {tmodel.reactions[idx].upper_bound} ----> {lb}, {ub}")

        tmodel.reactions[idx].upper_bound = ub
        tmodel.reactions[idx].lower_bound = lb
        

In [5]:
def clean_tfva_bounds(lb, ub, tol=1e-7):
    """
    Cleans numerical noise from TFVA bounds to improve solver stability.
    """
    old_lb = lb
    old_ub = ub
    if abs(lb) < tol: lb = 0.0
    if abs(ub) < tol: ub = 0.0
    
    if abs(lb - round(lb)) < tol: lb = round(lb, 8)
    if abs(ub - round(ub)) < tol: ub = round(ub, 8)
    
    if abs(ub - lb) < tol:
        avg = (lb + ub) / 2
        lb, ub = avg, avg

    if lb > ub:
        lb = ub 
    
    print(f"Before/after: {old_lb}, {old_ub} ---> {lb}, {ub}")

    return lb, ub

In [6]:
def setup_model(condition, mets_tva, flux_tfva):
    # Setup model with only removing blocked reactions
    # No compression or cofactor loops removed etc.
    tmodel = scripts.gen_model.gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, "", True, True)
    tmodel = scripts.gen_model.apply_physio_data(tmodel, condition, INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, allow_other_excr=False, output_log="", flux_limit=100)

    blocked_p = scripts.reaction_utils.list_blocked_reactions(tmodel, "core", "", 1, False)
    print(len(blocked_p))

    tmodel.remove_reactions(blocked_p, remove_orphans=True)
    scripts.metabolite_utils.remove_orphan_metabolites(tmodel)
    for rxn in tmodel.reactions:
        reaction_balance(rxn, balance_charge=True, balance_mg=False)
    tmodel.update_thermo_info(fit_unknown_dfG0=True)

    # set co2/o2 fluxes in correct directions
    tmodel.reactions.EX_co2.lower_bound = 0.0
    tmodel.reactions.EX_co2.upper_bound = 100.0
    
    tmodel.reactions.EX_o2.lower_bound = -100.0
    tmodel.reactions.EX_o2.upper_bound = 0.0

    scripts.metabolite_utils.apply_met_tva(tmodel, mets_tva)
    
    if flux_tfva is not None:
        apply_tfva_bounds(tmodel, flux_tfva)
        blocked_p = scripts.reaction_utils.list_blocked_reactions(tmodel, "core", "", 1, False)
        print(len(blocked_p))

        tmodel.remove_reactions(blocked_p, remove_orphans=True)
        tmodel.update_thermo_info(fit_unknown_dfG0=True)


    return tmodel

In [7]:
importlib.reload(scripts.metabolite_utils)

<module 'scripts.metabolite_utils' from '/home/laurens/MSC-Thermo/scripts/metabolite_utils.py'>

In [7]:
tmodel_glc = setup_model(CONDITIONS[0], "results/WT_Glc/ecoli_WT_Glc_mets.mps.gz_objval.txt", "results/WT_Glc/WT_Glc_objvals.txt")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2731723
Set parameter GURO_PAR_DUMP to value 1
Read parameters from file gurobi.env
Academic license 2731723 - for non-commercial use only - registered to l.___@student.rug.nl
['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' cre

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


Initializing component contribution object...
cxcalc is not installed, operating in read-only mode. A local cache may be loaded, but no compounds can be created. Install cxcalc and obtain a ChemAxon license to enable compound creation.
Loading compounds from ecoli_compound.sqlite
added reaction:  biomass_ce: biomass_c <=> biomass_e
added reaction:  biomass_EX: biomass_e <=> 
['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
biomass : 0.05 5mthf_c + 5e-05 accoa_c + 0.488 ala-L_c + 0.001 amp_c + 0.281 arg-L_c + 0.2
biomass_ce : biomass_c <=> biomass_e
biomass_EX : biomass_e <=> 
Identifying compounds...
[████████████████████████████████████████] 777/777 Mg_e                                           

Estimating dfG0'...
biomass_c 0 kilojoule / mole
biomass_e 0 kilojoule / mole
[███████

In [9]:
for met in tmodel_glc.metabolites:
    print(met.lower_bound, met.upper_bound)

9.999999998704212e-05 millimolar 1.0000000000000002 millimolar
0.00010000000001749102 millimolar 2.8331401300858796 millimolar
9.999999999661811e-05 millimolar 0.751009460922482 millimolar
0.0001000002340607334 millimolar 21.680209268635853 millimolar
0.00010000000005387791 millimolar 2.8197358694660406 millimolar
9.999999999999994e-05 millimolar 0.0693738923742067 millimolar
9.999999999987807e-05 millimolar 1.0000000000000002 millimolar
9.999999999986955e-05 millimolar 1.0000000000000002 millimolar
0.10000000000000009 millimolar 0.24709787183286175 millimolar
1000.0 millimolar 1000.0 millimolar
23.371565481269098 millimolar 25.0 millimolar
9.999999999162762e-05 millimolar 1.0000000000000002 millimolar
0.08564761062942244 millimolar 3.162955304435495 millimolar
9.999999999999994e-05 millimolar 1.0000000000000002 millimolar
9.9999999999936e-05 millimolar 1.0000000000000002 millimolar
1.7097749716836963 millimolar 44.59032086275668 millimolar
9.999999998497374e-05 millimolar 1.0000000000

In [8]:
tmodel_glc.m = None
tmodel_glc.objective = tmodel_glc.reactions.biomass_EX
tmodel_glc.add_TFBA_variables()

add_rq_constraint(tmodel_glc, "WT-Glc_I", INPUT_EXP_DATA)

Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10
>>>>>> RQ: 0.74 (vCO2 = 12.9 / vO2 = -17.4) ERR: 0.17932102098560615


In [26]:
# gas epsilon = 0.05
tmodel_glc.m.Params.TimeLimit = 3600
tmodel_glc.m.Params.NumericFocus = 1
tmodel_glc.m.optimize()

Set parameter TimeLimit to value 3600
Set parameter NumericFocus to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  3600
NonConvex  2
NumericFocus  1
GURO_PAR_DUMP  1

Academic license 2731723 - for non-commercial use only - registered to l.___@student.rug.nl
Optimize a model with 6431 rows, 5118 columns and 155011 nonzeros
Model fingerprint: 0xda34feed
Model has 1 quadratic constraint
Model has 1 simple general constraint
  1 NORM
Variable types: 4495 continuous, 623 integer (623 binary)
Coefficient statistics:
  Matrix range     [3e-06, 7e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-06, 1e+06]
  RHS range        [2e-14, 1e+05]
Presolve removed 3445 rows and 26

In [30]:
tmodel_glc.m.Params.TimeLimit = 3600
tmodel_glc.m.optimize()

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  3600
NonConvex  2
GURO_PAR_DUMP  1

Academic license 2731723 - for non-commercial use only - registered to l.___@student.rug.nl
Optimize a model with 6433 rows, 5119 columns and 155015 nonzeros
Model fingerprint: 0xde644267
Model has 1 quadratic constraint
Model has 1 simple general constraint
  1 NORM
Variable types: 4496 continuous, 623 integer (623 binary)
Coefficient statistics:
  Matrix range     [3e-06, 7e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e-01, 1e+06]
  RHS range        [2e-14, 1e+05]
Presolve removed 3143 rows and 2490 columns
Presolve time: 0.34s
Presolved: 3295 rows, 

In [ ]:
tmodel_ace = setup_model(CONDITIONS[1], "results/WT_Ace/ecoli_WT_Ace_mets.mps.gz_objval.txt", "results/WT_Ace/WT_Ace_objvals.txt")

['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' created
unknown metabolite '3hpppn[c]' created
unknown metabolite 'dhpppn[c]' created
unknown metabolite 'phthr[c]' created
unknown metabolite '4hthr[c]' created
unknown metabolite 'pi[c]' created
unknown metabolite '5dglcn[c]' created
unknown metabolite 'ru5p-D

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


ins_c NOTHING DONE!
nh3_c NOTHING DONE!
4adcho_c NOTHING DONE!
4abz_c NOTHING DONE!
chor_c NOTHING DONE!
gln-L_c NOTHING DONE!
ade_c NOTHING DONE!
hxan_c NOTHING DONE!
etoh_c NOTHING DONE!
gdp_c NOTHING DONE!
itp_c NOTHING DONE!
idp_c NOTHING DONE!
ametam_c NOTHING DONE!
camp_c NOTHING DONE!
adocbi_c NOTHING DONE!
rdmbzi_c NOTHING DONE!
adocbl_c NOTHING DONE!
gmp_c NOTHING DONE!
prpp_c NOTHING DONE!
aps_c NOTHING DONE!
paps_c NOTHING DONE!
dcamp_c NOTHING DONE!
fum_c NOTHING DONE!
25aics_c NOTHING DONE!
aicar_c NOTHING DONE!
asp-L_c NOTHING DONE!
imp_c NOTHING DONE!
acgam6p_c NOTHING DONE!
gam6p_c NOTHING DONE!
adphep-DD_c NOTHING DONE!
adphep-LD_c NOTHING DONE!
agm_c NOTHING DONE!
ptrc_c NOTHING DONE!
urea_c NOTHING DONE!
hcys-L_c NOTHING DONE!
rhcys_c NOTHING DONE!
10fthf_c NOTHING DONE!
fprica_c NOTHING DONE!
thf_c NOTHING DONE!
air_c NOTHING DONE!
5caiz_c NOTHING DONE!
5aizc_c NOTHING DONE!
ala-D_c NOTHING DONE!
alaala_c NOTHING DONE!
ala-L_c NOTHING DONE!
pydx5p_c NOTHING DONE!
py

In [8]:
tmodel_ace.m = None
tmodel_ace.objective = tmodel_ace.reactions.biomass_EX
tmodel_ace.add_TFBA_variables()

add_rq_constraint(tmodel_ace, "WT-Ace_I", INPUT_EXP_DATA)

NameError: name 'tmodel_ace' is not defined

In [14]:
tmodel_ace.m.Params.TimeLimit = 3600
tmodel_ace.m.optimize()

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  3600
NonConvex  2
GURO_PAR_DUMP  1

Academic license 2731723 - for non-commercial use only - registered to l.___@student.rug.nl
Optimize a model with 6428 rows, 5111 columns and 154775 nonzeros
Model fingerprint: 0x0d0850fb
Model has 1 quadratic constraint
Model has 1 simple general constraint
  1 NORM
Variable types: 4489 continuous, 622 integer (622 binary)
Coefficient statistics:
  Matrix range     [3e-06, 7e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e-02, 1e+06]
  RHS range        [2e-14, 1e+05]
Presolve removed 3143 rows and 2491 columns
Presolve time: 0.34s
Presolved: 3290 rows, 

In [44]:
# Write one model for all concentration TVA's scenarios
mets_glc = scripts.reaction_utils.tfva_run_scenarios_one_model_mets(tmodel_glc, "ecoli", "WT_Glc", "mets", "", False, False, ONLY_WRITE=True)
mets_ace = scripts.reaction_utils.tfva_run_scenarios_one_model_mets(tmodel_ace, "ecoli", "WT_Ace", "mets", "", False, False, ONLY_WRITE=True)

[]
0
0
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 99999
Set parameter MIPGap to value 0.001
Added 2dhglcn_c : <MVar ()>
array(<gurobi.Var ln_conc[0,0]>)
Added nadh_c : <MVar ()>
array(<gurobi.Var ln_conc[0,1]>)
Added glcn_c : <MVar ()>
array(<gurobi.Var ln_conc[0,2]>)
Added nad_c : <MVar ()>
array(<gurobi.Var ln_conc[0,3]>)
Added nadph_c : <MVar ()>
array(<gurobi.Var ln_conc[0,4]>)
Added nadp_c : <MVar ()>
array(<gurobi.Var ln_conc[0,5]>)
Added 2dhguln_c : <MVar ()>
array(<gurobi.Var ln_conc[0,6]>)
Added idon-L_c : <MVar ()>
array(<gurobi.Var ln_conc[0,7]>)
Added o2_c : <MVar ()>
array(<gurobi.Var ln_conc[0,8]>)
Added h2o_c : <MVar ()>
array(<gurobi.Var ln_conc[0,9]>)
Added pi_c : <MVar ()>
array(<gurobi.Var ln_conc[0,10]>)
Added 5dglcn_c : <MVar ()>
array(<gurobi.Var ln_conc[0,11]>)
Added ru5p-D_c : <MVar ()>
array(<gurobi.Var ln_conc[0,12]>)
Added ara5p_c : <MVar ()>
array(<gurobi.Var ln_conc[0,13]>)
Added ACP_c : <MVar ()>

In [15]:
# Write one mps file containign 2 scenarios for each reaction for the models
scripts.reaction_utils.tfva_run_scenarios_one_model(tmodel_glc, "ecoli", "WT_Glc", "results/WT_Glc/parallel/", REMOVE_BLOCKED=False, APPLY_FVA=False, RUN_DIRECTLY=True, ONLY_WRITE=True, mipgap=0.01)
scripts.reaction_utils.tfva_run_scenarios_one_model(tmodel_ace, "ecoli", "WT_Ace", "results/WT_Ace/parallel/", REMOVE_BLOCKED=False, APPLY_FVA=False, RUN_DIRECTLY=True, ONLY_WRITE=True, mipgap=0.01)

[]
0
0
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 99999
Set parameter MIPGap to value 0.01
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 99999
Set parameter MIPGap to value 0.01
Running reaction: 2DGLCNRx: 2dhglcn_c + h_c + nadh_c <=> glcn_c + nad_c, 0, <MVar ()>
array(<gurobi.Var fluxes[0,0]>)
0
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 99999
Set parameter MIPGap to value 0.01
Running reaction: 2DGLCNRy: 2dhglcn_c + h_c + nadph_c <=> glcn_c + nadp_c, 1, <MVar ()>
array(<gurobi.Var fluxes[0,1]>)
0
Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10
Set parameter TimeLimit to value 99999
Set parameter MIPGap to value 0.01
Running reaction: 2DGULRx: 2dhguln_c + h_c + nadh_c <=> idon-L_c + nad_c, 2, <MVar ()>
array(<gurobi.Var fluxes[0,2]>)
0
Set parameter NonConvex to value 2
Set parameter T

In [9]:
import scripts.tfs.preprocess
import scripts.tfs.thermospace_mod
import scripts.tfs.tbasis_mod
import glob

importlib.reload(scripts.tfs.preprocess)
importlib.reload(scripts.tfs.thermospace_mod)
importlib.reload(scripts.tfs.tbasis_mod)

def tfs_preprocess(tfs_dir, ranktol = 1e-5):

    tfs_files_dir = glob.glob(f"{tfs_dir}/*.*")
    print(tfs_files_dir)
    model_file = [x for x in tfs_files_dir if x.endswith(".sbml")][0]
    vbounds = [x for x in tfs_files_dir if x.endswith("vbounds.csv")][0]
    lcv = [x for x in tfs_files_dir if x.endswith("lcv.csv")][0]
    lcm = [x for x in tfs_files_dir if x.endswith("lcm.csv")][0]
    lcb = [x for x in tfs_files_dir if x.endswith("lcb.csv")][0]
    drg0pm = [x for x in tfs_files_dir if x.endswith("drg0pm.csv")][0]
    drg0cs = [x for x in tfs_files_dir if x.endswith("drg0cs.csv")][0]

    prep = scripts.tfs.preprocess.Preprocess(
        cobra_file=model_file,
        vbound_file=vbounds,
        logconcmean_file=lcm,
        logconcvar_file=lcv,
        lncbounds_file=lcb,
        drG0file=drg0pm,
        drGcovsqrtfile=drg0cs,
        drG0covfile=None#"tfs/drg0c_test.csv",
    )

    thermodynamic_space = scripts.tfs.thermospace_mod.ThermodynamicSpaceMod(
        prep.Sconstrained,
        prep.rid_constrained,
        prep.model.metabolites,
        prep.drg0_prime_mean,
        prep.drg0_prime_cov_sqrt,
        prep.log_conc_cov,
        prep.log_conc_mean,
    )


    tbasis = scripts.tfs.tbasis_mod.ThermodynamicSpaceBasismod(
        thermodynamic_space,
        explicit_log_conc=False,
        explicit_drg0=False,
        explicit_drg=True,
        min_eigenvalue=1e-10,
        ranktol=ranktol,
        #ignoreconc_met_idx=prep.ignoreconc_met_idx,
    )

    return (prep, thermodynamic_space, tbasis)


In [38]:
scripts.gen_model.prepare_tfs_files(tmodel_glc, "ecoli", "WT_Glc", "results/WT_Glc/tfs/")

In [18]:
tmodel = scripts.gen_model.gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, "", True, True)
tmodel = scripts.gen_model.apply_physio_data(tmodel, CONDITIONS[0], INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, allow_other_excr=False, output_log="", flux_limit=100)

blocked_p = scripts.reaction_utils.list_blocked_reactions(tmodel, "core", "", 1, False)
print(len(blocked_p))

tmodel.remove_reactions(blocked_p, remove_orphans=True)
scripts.metabolite_utils.remove_orphan_metabolites(tmodel)
for rxn in tmodel.reactions:
    reaction_balance(rxn, balance_charge=True, balance_mg=False)
tmodel.update_thermo_info(fit_unknown_dfG0=True)


['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' created
unknown metabolite '3hpppn[c]' created
unknown metabolite 'dhpppn[c]' created
unknown metabolite 'phthr[c]' created
unknown metabolite '4hthr[c]' created
unknown metabolite 'pi[c]' created
unknown metabolite '5dglcn[c]' created
unknown metabolite 'ru5p-D

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


*** Updating metabolite information ***
2dhglcn_c NOTHING DONE!
nadh_c NOTHING DONE!
glcn_c NOTHING DONE!
nad_c NOTHING DONE!
nadph_c NOTHING DONE!
nadp_c NOTHING DONE!
2dhguln_c NOTHING DONE!
idon-L_c NOTHING DONE!
3hcinnm_c NOTHING DONE!
o2_c NOTHING DONE!
dhcinnm_c NOTHING DONE!
h2o_c NOTHING DONE!
3hpppn_c NOTHING DONE!
dhpppn_c NOTHING DONE!
phthr_c NOTHING DONE!
4hthr_c NOTHING DONE!
pi_c NOTHING DONE!
5dglcn_c NOTHING DONE!
ru5p-D_c NOTHING DONE!
ara5p_c NOTHING DONE!
ACP_c NOTHING DONE!
atp_c NOTHING DONE!
ttdca_c NOTHING DONE!
amp_c NOTHING DONE!
myrsACP_c NOTHING DONE!
ppi_c NOTHING DONE!
ttdcea_c NOTHING DONE!
tdeACP_c NOTHING DONE!
hdca_c NOTHING DONE!
palmACP_c NOTHING DONE!
hdcea_c NOTHING DONE!
hdeACP_c NOTHING DONE!
ocdcea_c NOTHING DONE!
octeACP_c NOTHING DONE!
dtdp4aaddg_c NOTHING DONE!
unagamu_c NOTHING DONE!
dtdp_c NOTHING DONE!
unagamuf_c NOTHING DONE!
arbt6p_c NOTHING DONE!
g6p_c NOTHING DONE!
hqn_c NOTHING DONE!
4abut_c NOTHING DONE!
akg_c NOTHING DONE!
glu-L_c N

In [19]:
cobra.io.write_sbml_model(tmodel, "tfs/WT_Glc_I/test.sbml")

In [20]:
glc_cobra = cobra.io.read_sbml_model("tfs/WT_Glc_I/test.sbml")
#glc_cobra.objective = glc_cobra.reactions.biomass_EX

ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


In [ ]:
import pta
importlib.reload(pta)
pta.prepare_for_pta(glc_cobra, biomass_id="biomass_EX", atpm_id="ATPS4r")

Read LP format model from file /tmp/tmp3neffxk9.lp
Reading time = 0.00 seconds
: 454 rows, 1246 columns, 6358 nonzeros
2026-01-18  18:24:31.060  main                     INFO     | logger initialized
2026-01-18  18:24:31.061  main                     INFO     | =====================================================
2026-01-18  18:24:31.061  main                     INFO     | efmtool version 4.7.1, 2009-12-04 18:30:05
2026-01-18  18:24:31.062  main                     INFO     | Copyright (c) 2009, Marco Terzer, Zurich, Switzerland
2026-01-18  18:24:31.062  main                     INFO     | This is free software, !!! NO WARRANTY !!!
2026-01-18  18:24:31.062  main                     INFO     | See LICENCE.txt for redistribution conditions
2026-01-18  18:24:31.062  main                     INFO     | =====================================================
2026-01-18  18:24:31.220  main    efm.output.mat   INFO     | estimated efms-per-file: 590000
2026-01-18  18:24:31.237  main    efm.im

In [ ]:
tfs_data = tfs_preprocess("results/WT_Glc/tfs/")

['results/WT_Glc/tfs/ecoli_WT_Glc_vbounds.csv', 'results/WT_Glc/tfs/ecoli_WT_Glc_lcb.csv', 'results/WT_Glc/tfs/ecoli_WT_Glc_drg0cs.csv', 'results/WT_Glc/tfs/ecoli_WT_Glc_drg0pm.csv', 'results/WT_Glc/tfs/ecoli_WT_Glc_lcm.csv', 'results/WT_Glc/tfs/ecoli_WT_Glc_lcv.csv', 'results/WT_Glc/tfs/ecoli_WT_Glc.sbml']


ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


620
[]
[616, 520, 615, 514, 513, 522, 619, 564, 521, 519, 515, 517, 516, 518]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 2

In [56]:
import pta
problem = pta.PmoProblem(tfs_data[0].model, tfs_data[1], tfs_data[2], solver="GUROBI")

In [58]:
import scripts.tfs.tfsmodel_mod
importlib.reload(scripts.tfs.tfsmodel_mod)

from pta.sampling.tfs import (
    TFSModel,
    sample_drg,
    sample_drg0_from_drg,
    sample_log_conc_from_drg,
    sample_fluxes_from_drg
)

pta.enable_all_logging()

tfs_model = scripts.tfs.tfsmodel_mod.SupTFSmodel(tfs_data[0].model, tfs_data[1], tfs_data[2], problem, solver="GUROBI")
result = sample_drg(tfs_model)

print(result)

Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.



Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received

Interrupt request received


Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.


KeyboardInterrupt: 

In [ ]:
# OLD

import pandas as pd

def add_gas_constraint(model, condition, phys_data, epsilon=0.25):

    reg_data = pd.read_csv(phys_data)

    reg_data.set_index(["cond", "rxn"], inplace=True) 

    reg_data_gas = reg_data.swaplevel().copy()
    reg_data_gas = reg_data_gas.loc[["EX_co2", "EX_o2"]]
    reg_data_gas = reg_data_gas.swaplevel()

    co2_mean = reg_data_gas.loc[(condition, "EX_co2"), "mean"]
    o2_mean = reg_data_gas.loc[(condition, "EX_o2"), "mean"]

    co2_sd = reg_data_gas.loc[(condition, "EX_co2"), "sd"]
    o2_sd = reg_data_gas.loc[(condition, "EX_o2"), "sd"]


    if model.m is not None:
        co2_idx = model.reactions.index("EX_co2")
        o2_idx = model.reactions.index("EX_o2")

        # as we have a mean and sd for the o2/co2 fluxes I am using a ratio range corresponding to the lower/upper range of the mean + sd
        R_upper = abs( (co2_mean + co2_sd) / (o2_mean + o2_sd) )
        R_lower = abs( (co2_mean - co2_sd) / (o2_mean - o2_sd) )

        # add constraints for the lower range of the ratio and upper range of the ratio
        # also, instead of saying that it needs to be 0, I set it to be within a specific range (the epsilon)
        # I noticed that the model becomes infeasible for Glc/Ace with an epsilon of 0.05 (5%), with 0.25 (25%) it is feasible
        model.m.addConstr( model.mvars["v"][0][co2_idx] + (R_lower * model.mvars["v"][0][o2_idx]) >= -epsilon, name=f"RQ_ratio_lower_{condition}")
        model.m.addConstr( model.mvars["v"][0][co2_idx] + (R_upper * model.mvars["v"][0][o2_idx]) <= epsilon, name=f"RQ_ratio_upper_{condition}")
    else:
        "No gurobi model, add TFBA variables first."